# 23. 무관찰 추리 (ReWoo)

In [1]:
!pip install -U langgraph langchain_community langchain_openai tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found

In [2]:
import getpass
import os

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}=")

_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("OPENAI_API_KEY")


TAVILY_API_KEY=··········
OPENAI_API_KEY=··········


In [3]:
from typing import List
from typing_extensions import TypedDict

class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str


In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")
prompt = """다음 과업에 대해, 문제를 단계별로 해결할 수 있는 일련의 계획을 작성하라. 각 계획마다 어떤 외부 도구와 그에 해당하는 입력을 사용해 증거를 수집할지 명시하라. 증거는 이후 도구들이 참조할 수 있는 변수 #E (예: #E1, #E2, #E3 등)에 저장할 수 있다. 모든 변수는 독립적이므로 각 도구 입력에 필요한 모든 정보를 포함해야 한다.

도구는 다음 중 하나일 수 있다.

Google[input]: 구글에서 결과를 검색하는 검색 엔진 작업자다. 간결한 답변이나 특정 주제에 대한 정보를 필요로 할 때 사용하라. 입력은 검색 쿼리여야 한다.

LLM[input]: 사전 학습된 대규모 언어 모델(나와 같은)이다. 일반적인 세계 지식, 상식, 또는 복잡한 추리를 활용해야 할 때 사용하라. 외부 도움 없이 문제를 해결할 자신이 있을 때 이 도구를 우선 사용하라. 입력은 어떤 지시나 질문이 될 수 있다.

Calculator[input]: 수학적 계산을 수행할 수 있는 도구다. 산술 연산을 수행해야 할 때 사용하라. 입력은 유효한 수학적 표현식이어야 한다.

WolframAlpha[input]: 계산 지식 엔진이다. 방정식을 풀거나 기호적 계산을 수행하거나 데이터 기반의 답변을 얻어야 할 때 사용하라. 입력은 수학 또는 과학 문제와 관련된 Wolfram 언어나 자연어 쿼리여야 한다.

예를 들어,
과업: 앨리스, 밥, 캐롤이 지난주 아르바이트로 총 540달러를 벌었다. 앨리스는 y 달러를 벌었다. 밥은 앨리스가 번 돈의 3배보다 20달러 더 벌었고, 캐롤은 밥보다 15달러를 더 벌었다. 캐롤이 번 돈은 얼마인가?

계획: 앨리스가 y 달러를 벌었다고 가정하고, 문제를 대수적 표현으로 변환해 Wolfram Alpha로 푼다.
#E1 = WolframAlpha[Solve y + (3y + 20) + ((3y + 20) + 15) = 540]

계획: 앨리스가 얼마 벌었는지 알아낸다.
#E2 = LLM[What is y, given #E1]

계획: 캐롤이 얼마 벌었는지 계산한다.
#E3 = Calculator[((3 * #E2) + 20) + 15]

시작!
계획을 풍부한 세부 사항으로 설명해라. 각 계획은 하나의 #E로만 끝나야 한다.

과업: {task}"""


In [10]:
import re
from langchain_core.prompts import ChatPromptTemplate

regex_pattern = (
    r"계획:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*"
    r"\[([^\]]+)\]"
)
prompt_template = ChatPromptTemplate.from_messages(
    [("user", prompt)]
)
planner = prompt_template | model

def get_plan(state: ReWOO):
    task = state["task"]
    result = planner.invoke({"task": task})
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}


In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1

def tool_execution(state: ReWOO):
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}

    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)

    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError

    _results[step_name] = str(result)
    return {"results": _results}


In [12]:
solve_prompt = """다음 과업 또는 문제를 해결하라. 문제를 해결하기 위해 단계별 계획을 세우고 각 계획에 해당하는 증거를 검색했다. 긴 증거는 관련 없는 정보를 포함할 수 있으니 주의해서 사용하라.

{plan}

이제 위에 제공된 증거에 따라 질문이나 과업을 해결하라. 추가적인 말 없이 직접 답변하라.

과업: {task}
응답:"""

def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (
            (state["results"] or {}) if "results" in state else {}
        )
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += (
            f"계획: {_plan}\n"
            f"{step_name} = {tool}[{tool_input}]\n"
        )

    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}


In [13]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        return "solve"
    else:
        return "tool"

from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()


In [14]:
task = "2024년 호주 오픈 남자 단식 우승자의 정확한 고향은 어디인가"
for s in app.stream({"task": task}):
    print(s)
    print("---")


{'plan': {'steps': [('2024년 호주 오픈 남자 단식 우승자가 누구인지 알아본다. 이 정보를 통해 그의 고향을 찾을 수 있다.', '#E1', 'Google', '"2024 호주 오픈 남자 단식 우승자"'), ('우승자의 고향을 확인하기 위해 더 많은 정보를 수집한다. #E1에서 우승자의 이름이 있을 것이므로, 이를 사용해 고향을 검색한다.', '#E2', 'Google', '"#E1 고향"')], 'plan_string': '계획: 2024년 호주 오픈 남자 단식 우승자가 누구인지 알아본다. 이 정보를 통해 그의 고향을 찾을 수 있다.\n#E1 = Google["2024 호주 오픈 남자 단식 우승자"]\n\n계획: 우승자의 고향을 확인하기 위해 더 많은 정보를 수집한다. #E1에서 우승자의 이름이 있을 것이므로, 이를 사용해 고향을 검색한다.\n#E2 = Google["#E1 고향"]\n\n이 계획을 통해 2024년 호주 오픈 남자 단식 우승자의 정확한 고향을 알아낼 수 있을 것이다.'}}
---
{'tool': {'results': {'#E1': '[{\'title\': "2024 Australian Open – Men\'s singles final - Wikipedia", \'url\': \'https://en.wikipedia.org/wiki/2024_Australian_Open_%E2%80%93_Men%27s_singles_final\', \'content\': \'The 2024 Australian Open Men\\\'s Singles final was the championship tennis match of the men\\\'s singles tournament at the 2024 Australian Open, contested by fourth-seed Jannik Sinner and third-seed Daniil Medvedev. Sinner came back to defeat Medvedev from two-sets